## 수작업으로 변수 선택. 
### 명세서가 부실한 변수와 NA가 많은 변수는 제거

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

### 분기별 사고/사람/차량으로 각각 데이터 통합

In [2]:
# 데이터 load하고 append하는데 다소 시간이 소요될 수 있습니다.
accident_data = []
person_data = []
vehicle_data = []
for q in [1,2,3,4]:
    accident_data.append(pd.read_excel('data/Crash_Qtr0'+str(q)+'_2015.xlsx', sheetname = 0))
    person_data.append(pd.read_excel('data/Crash_Qtr0'+str(q)+'_2015.xlsx', sheetname = 1))
    vehicle_data.append(pd.read_excel('data/Crash_Qtr0'+str(q)+'_2015.xlsx', sheetname = 2))

In [3]:
accidents = pd.concat(accident_data).reset_index(drop=True)
persons = pd.concat(person_data).reset_index(drop=True)
vehicles = pd.concat(vehicle_data).reset_index(drop=True)

#### 각 영역별 데이터 타입 확인

In [4]:
accidents.dtypes

ACC_DATE                object
ACC_TIME                object
AGENCY_CODE             object
AREA_CODE               object
COLLISION_TYPE_CODE      int64
COUNTY_NO              float64
C_M_ZONE_FLAG           object
DISTANCE               float64
DISTANCE_DIR_FLAG       object
FEET_MILES_FLAG         object
FIX_OBJ_CODE           float64
HARM_EVENT_CODE1       float64
HARM_EVENT_CODE2       float64
JUNCTION_CODE          float64
LANE_CODE              float64
LATITUDE               float64
LIGHT_CODE             float64
LOC_CODE                object
LOGMILE_DIR_FLAG        object
LOG_MILE               float64
LONGITUDE              float64
MAINROAD_NAME           object
MUNI_CODE              float64
RD_COND_CODE           float64
RD_DIV_CODE            float64
REFERENCE_NO           float64
REFERENCE_ROAD_NAME     object
REFERENCE_SUFFIX        object
REFERENCE_TYPE_CODE     object
REPORT_NO               object
ROUTE_TYPE_CODE         object
RTE_NO                 float64
RTE_SUFF

In [5]:
persons.dtypes

ALCOHOL_TESTTYPE_CODE    float64
ALCOHOL_TEST_CODE        float64
BAC_CODE                 float64
CDL_FLAG                  object
CLASS                     object
CONDITION_CODE           float64
DATE_OF_BIRTH             object
DRUG_TESTRESULT_CODE      object
DRUG_TEST_CODE           float64
EJECT_CODE               float64
EMS_UNIT_LABEL            object
EQUIP_PROB_CODE          float64
FAULT_FLAG                object
INJ_SEVER_CODE             int64
LICENSE_STATE_CODE        object
MOVEMENT_CODE            float64
OCC_SEAT_POS_CODE        float64
PED_LOCATION_CODE        float64
PED_OBEY_CODE            float64
PED_TYPE_CODE            float64
PED_VISIBLE_CODE         float64
PERSON_ID                 object
PERSON_TYPE               object
REPORT_NO                 object
SAF_EQUIP_CODE           float64
SEX_CODE                  object
VEHICLE_ID                object
dtype: object

In [6]:
vehicles.dtypes

AREA_DAMAGED_CODE1           float64
AREA_DAMAGED_CODE2           float64
AREA_DAMAGED_CODE3           float64
AREA_DAMAGED_CODE_IMP1       float64
AREA_DAMAGED_CODE_MAIN       float64
BODY_TYPE_CODE               float64
COMMERCIAL_FLAG               object
CONTI_DIRECTION_CODE          object
CV_BODY_TYPE_CODE            float64
DAMAGE_CODE                    int64
DRIVERLESS_FLAG               object
FIRE_FLAG                     object
GOING_DIRECTION_CODE          object
GVW_CODE                     float64
HARM_EVENT_CODE              float64
HAZMAT_SPILL_FLAG             object
HIT_AND_RUN_FLAG              object
HZM_NUM                       object
MOVEMENT_CODE                float64
NUM_AXLES                    float64
PARKED_FLAG                   object
REPORT_NO                     object
SPEED_LIMIT                  float64
TOWED_AWAY_FLAG               object
TOWED_VEHICLE_CONFIG_CODE    float64
VEHICLE_ID                    object
VEH_MAKE                      object
V

#### 차량과 사람데이터에서 동일한 column name인 MOVEMENT_CODE 존재
#### 다음 단계에서 불필요한 변수를 제거하고자 할 때, column name이 중복되어 미리 제거

In [7]:
persons.drop("MOVEMENT_CODE", axis=1, inplace=True)

### 사고/사람/차량 데이터 통합(merge)

In [8]:
merged_data = pd.merge(persons, accidents, how='inner',on='REPORT_NO')
merged_data = pd.merge(merged_data, vehicles, how='left',on="VEHICLE_ID")
merged_data.drop('REPORT_NO_y', axis=1, inplace=True)
merged_data.rename(columns = {'REPORT_NO_x' : 'REPORT_NO'}, inplace = True)

### 분석에 필요하다고 판단되는 변수만 남김

In [9]:
necessary_columns_for_analysis = ['CDL_FLAG', 'CONDITION_CODE', 'EQUIP_PROB_CODE', 'FAULT_FLAG',
       'INJ_SEVER_CODE', 'PERSON_ID', 'PERSON_TYPE', 'REPORT_NO',
       'SAF_EQUIP_CODE', 'SEX_CODE', 'VEHICLE_ID', 'COLLISION_TYPE_CODE',
       'C_M_ZONE_FLAG', 'JUNCTION_CODE', 'LANE_CODE', 'LIGHT_CODE',
       'RD_COND_CODE', 'RD_DIV_CODE', 'SURF_COND_CODE', 'WEATHER_CODE',
       'AREA_DAMAGED_CODE_MAIN', 'BODY_TYPE_CODE', 'DAMAGE_CODE',
       'HIT_AND_RUN_FLAG', 'MOVEMENT_CODE', 'ACC_DATE', 'ACC_TIME', 'DATE_OF_BIRTH']

In [10]:
merged_data = merged_data[necessary_columns_for_analysis]

## 사고 발생 일시 관련 변수 통합

In [11]:
merged_data['TIME'] = pd.to_datetime(merged_data.ACC_DATE + ' ' + merged_data.ACC_TIME)
merged_data.drop('ACC_DATE', axis=1, inplace=True)
merged_data.drop('ACC_TIME', axis=1, inplace=True)

## 나이 변수 생성. (Noise 제거를 위해 16세 이상, 90세 이하만 분석)

In [12]:
merged_data['AGE'] = merged_data.DATE_OF_BIRTH.apply(lambda x : 100 - int(str(x)[-2:]) + 15 if type(x) == str else x)
merged_data = merged_data.loc[np.array(merged_data.AGE >= 16) & np.array(merged_data.AGE <= 90)]
merged_data.drop('DATE_OF_BIRTH', axis=1, inplace=True)

In [13]:
merged_data.shape

(200118, 27)

## NA가 포함된 column, 해당 변수로 의미를 알 수 없는 변수(0,88,99,U) 제거

In [14]:
merged_data = merged_data.dropna()
merged_data = merged_data.reset_index(drop = True)

cols = merged_data.columns

In [33]:
code_list = []
cate_list = []
for col_index in range(len(cols)):
    for i in merged_data[cols[col_index]].unique():
        if i in [0,88,99]:
            code_list.append(col_index)
            break
        if i == 'U':
            cate_list.append(col_index)
            break

C:\Users\HyoChang\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


In [45]:
index_0 = np.sum(merged_data[code_list] == 0, axis=1) != 0
index_88 = np.sum(merged_data[code_list] == 88, axis=1) != 0
index_99 = np.sum(merged_data[code_list] == 99, axis=1) != 0
index_U = np.sum(merged_data[cate_list] == 'U',axis = 1) != 0

In [46]:
i = 0
del_list = []
for x,y,z,w in zip(index_0, index_88, index_99, index_U):
    if x or y or z or w:
        del_list.append(i)
    i += 1

In [47]:
merged_data = merged_data.drop(merged_data.index[del_list])
merged_data = merged_data.reset_index(drop=True)

#### CODE BOOK에 JUNCTION_CODE 11.04이 누락되어 있어서 제거

In [48]:
del_junc_code_11 = list(merged_data.loc[merged_data['JUNCTION_CODE']==11.04].index)

In [49]:
merged_data = merged_data.drop(merged_data.index[del_junc_code_11])
merged_data = merged_data.reset_index(drop=True)

In [50]:
merged_data.shape

(47400, 27)

## code화 되어있는 변수를 변수값 코드북 참조해서 전환
#### ex) 기존에 LIGHT_CODE column의 값이 3이었다면 이를 Dark Lights On으로 변경

In [51]:
desc = pd.read_csv('desc.csv', header=None)

In [52]:
desc

,0,1,2
0,LIGHT_CODE,0.00,Not Applicable
1,LIGHT_CODE,1.00,Daylight
2,LIGHT_CODE,3.00,Dark Lights On
3,LIGHT_CODE,4.00,Dark No Lights
4,LIGHT_CODE,5.02,Dawn
5,LIGHT_CODE,6.02,Dusk
6,LIGHT_CODE,7.88,Dark - Unknown Lighting
7,LIGHT_CODE,88.00,Other
8,LIGHT_CODE,99.00,Unknown
9,JUNCTION_CODE,0.00,Not Applicable


In [53]:
# column namep에서 'CODE'가 포함된 경우 변환 필요. (SEX_CODE제외)
cols = merged_data.columns
change_list = []
for i in range(len(merged_data.columns)):
    if "CODE" in cols[i]:
        if cols[i] != "SEX_CODE":
            change_list.append(cols[i])

In [54]:
column_and_code = []
desc_list = []
for i in range(len(desc)):
    row = desc.loc[i]
    column_and_code.append(str(row[0])+'_'+str(row[1]))
    desc_list.append(str(row[2]))    

In [56]:
dict_ = {}
for i in range(len(column_and_code)):
    dict_[column_and_code[i]] = desc_list[i]

In [57]:
def code2desc(column_name, code):
    key = str(column_name)+'_'+str(code)
    return dict_[key]

In [59]:
for i in tqdm(change_list):
    merged_data[i] = merged_data[i].apply(lambda x: code2desc(i, float(x)))

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 19.33it/s]


In [72]:
merged_data.to_csv('maryland_accident_dataset.csv', index=False)